In [1]:
from os import listdir
from matplotlib.pyplot import imread,imshow,figure,show
from skimage.color import rgb2hsv
from skimage.color import rgb2gray
import numpy as np
from skimage import feature as sk_feature
from scipy.spatial.distance import euclidean
import operator

In [2]:
def color_features(image):
    hsv = rgb2hsv(imread(image))
    R = hsv[:,:,0]
    G = hsv[:,:,1]
    B = hsv[:,:,2]
    feature = [np.mean(R[:]),np.std(R[:]),np.mean(G[:]),np.std(G[:]),np.mean(B[:]),np.std(B[:])]
    return feature

# feature = color_features("dataset/382001.jpg")
# print(feature)

In [3]:
def histogram_features(image):
    hsv = rgb2hsv(imread(image))
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    v = hsv[:,:,2]

    feature = np.zeros((8,2,2))

    h_max = h.max()
    s_max = s.max()
    v_max = v.max()

    h_quantized = np.ceil((8 * h)/h_max);
    s_quantized = np.ceil((2 * s)/s_max);
    v_quantized = np.ceil((2 * v)/v_max);
    
    dimensions = hsv.shape
    
    index = np.zeros((dimensions[0]*dimensions[1],3))
    index[:,0] = h_quantized.T.flatten()
    index[:,1] = s_quantized.T.flatten()
    index[:,2] = v_quantized.T.flatten()

    for row in range(0,dimensions[0]*dimensions[1]):
        if(index[row][0]==0 or index[row][1]==0 or index[row][2]==0):
            continue
        feature[int(index[row][0])-1][int(index[row][1])-1][int(index[row][2])-1]=feature[int(index[row][0])-1][int(index[row][1])-1][int(index[row][2])-1]+1

    return feature.flatten().tolist()

# feature = histogram_features("dataset/382001.jpg")
# print(feature

In [4]:
def texture_features(image):
    image = np.array(rgb2gray(imread(image)), int)
    
    glcm = sk_feature.greycomatrix(image,distances=[0,1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],levels=255 , symmetric=True, normed=True)
    
    feature = np.concatenate((sk_feature.greycoprops(glcm, 'contrast'),sk_feature.greycoprops(glcm, 'correlation'),sk_feature.greycoprops(glcm, 'energy'),sk_feature.greycoprops(glcm, 'homogeneity')))
    feature = feature/np.sum(feature)
    
    return feature.flatten().tolist()

# feature = texture_features("dataset/382001.jpg")
# print(feature)

In [5]:
def getFeatures(path,fsize):
    features = []
    etats = []
    
    dirs = listdir(path)
    for _dir in dirs:
        images = listdir(path + "/" + _dir)
        for image in images:
            feature = []
            if fsize >= 6:
                color_feature = color_features(path + "/" + _dir + "/" +  image)
                feature += color_feature
            if fsize >= 32:
                histogram_feature = histogram_features(path + "/" + _dir + "/" +  image)
                feature += histogram_feature
            if fsize >= 36:
                texture_feature = texture_features(path + "/" + _dir + "/" +  image)
                feature += texture_feature
            features.append(feature)
            etats.append(_dir)
            
    return features,etats 
    
# feature = getFeatures("DB2C",32)
# print(feature)

In [6]:
from sklearn import svm
from sklearn.model_selection import train_test_split
import random

fsize = 32
path = "DB2C"

features,etats = getFeatures(path,fsize)

(trainFeat, testFeat, trainEtat, testEtat) = train_test_split(np.array(features), np.array(etats), test_size=0.1, random_state=random.seed())
clf = svm.SVC(gamma=0.01, C=100)
clf.fit(trainFeat,trainEtat)
print("taux de précision: ",clf.score(testFeat,testEtat))

taux de précision:  0.8367346938775511


In [7]:
from sklearn import tree

features,etats = getFeatures(path,fsize)

clf=tree.DecisionTreeClassifier()
(trainFeat, testFeat, trainEtat, testEtat) = train_test_split(np.array(features), np.array(etats), test_size=0.1, random_state=random.seed())
clf.fit(trainFeat,trainEtat)
print("taux de précision: ",clf.score(testFeat,testEtat))

taux de précision:  0.8367346938775511


In [8]:
def prediction(image,fsize):
        feature = []
        if fsize >= 6:
            color_feature = color_features(image)
            feature += color_feature
        if fsize >= 32:
            histogram_feature = histogram_features(image)
            feature += histogram_feature
        if fsize >= 36:
            texture_feature = texture_features(image)
            feature += texture_feature

        print(image,clf.predict([feature]))
        
prediction("DataToPredict/535091.jpg",32)

DataToPredict/535091.jpg ['obj_ship']
